In [1]:
import requests
import json 
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

# Etapa 1 - Dados Gerais

# Treatment

In [2]:
#Vamos dar uma olhada em todas as contas que hoje estão ativas
events = requests.get("https://graph.facebook.com/v10.0/me?fields=adaccounts%7Bname%7D&access_token=EAAXwBOcBhNIBAOmxrPUCMTcDNbO8ZB8JQj8VUlCDUZBZADiQyR7wBrqizZAgkJcQqhuoZCFs7KdIhxHtJVCVBZACyAxzEzckWnwkIGZBPs27oEe6psbx3DrHxeaKNnlCZBC3aZCelDAEBzM42Ed9Jy1ptvzFhr1Hm3ZBluZCvyL1vXcVQZDZD")
trans = json.loads(events.content)
contas = pd.json_normalize(trans['adaccounts']['data'])

In [3]:
#Fazendo requisicao para graphi face api para listar todas as campanhas existentes
#As campanhas mais utilizadas são de Branding e Redes, então teremos que realizar o tratamento duas vezes para cada uma
# Redes : act_625898984510034
# Branding: act_744828579330411

events = requests.get("https://graph.facebook.com/v10.0/act_625898984510034/campaigns?fields=id%2Cname%2Cdaily_budget%2Cbid_strategy&time_increment=1&date_preset=this_year&access_token=EAAXwBOcBhNIBAOmxrPUCMTcDNbO8ZB8JQj8VUlCDUZBZADiQyR7wBrqizZAgkJcQqhuoZCFs7KdIhxHtJVCVBZACyAxzEzckWnwkIGZBPs27oEe6psbx3DrHxeaKNnlCZBC3aZCelDAEBzM42Ed9Jy1ptvzFhr1Hm3ZBluZCvyL1vXcVQZDZD")
trans = json.loads(events.content)
redes = pd.json_normalize(trans['data'])

events = requests.get("https://graph.facebook.com/v10.0/act_744828579330411/campaigns?fields=id%2Cname%2Cdaily_budget%2Cbid_strategy&time_increment=1&date_preset=this_year&access_token=EAAXwBOcBhNIBAOmxrPUCMTcDNbO8ZB8JQj8VUlCDUZBZADiQyR7wBrqizZAgkJcQqhuoZCFs7KdIhxHtJVCVBZACyAxzEzckWnwkIGZBPs27oEe6psbx3DrHxeaKNnlCZBC3aZCelDAEBzM42Ed9Jy1ptvzFhr1Hm3ZBluZCvyL1vXcVQZDZD")
trans = json.loads(events.content)
branding = pd.json_normalize(trans['data'])

Abaixo vamos fazer o tratamento dos dados das campanhas de Redes

In [4]:
#Requisitando as informações via url retirado do graph facebook api
events = requests.get("https://graph.facebook.com/v10.0/act_625898984510034/campaigns?fields=insights&time_increment=1&date_preset=this_year&access_token=EAAXwBOcBhNIBAOmxrPUCMTcDNbO8ZB8JQj8VUlCDUZBZADiQyR7wBrqizZAgkJcQqhuoZCFs7KdIhxHtJVCVBZACyAxzEzckWnwkIGZBPs27oEe6psbx3DrHxeaKNnlCZBC3aZCelDAEBzM42Ed9Jy1ptvzFhr1Hm3ZBluZCvyL1vXcVQZDZD")
#A base vai vir com campaigns_id que não possui dados internos. Tratamento dos dados abaixo
trans = json.loads(events.content)
trans2 = pd.json_normalize(trans['data']) #os dados que queremos é insights.data
trans2 = trans2.dropna(subset=['insights.data'],inplace=False).reset_index() #removendo as linhas nulas
del trans2['index'] #tratando index

#Agora vamos criar um loop para preencher uma tabela com os dados que queremos
size = trans2['id'].count()

dataset2 = pd.DataFrame(columns=['account_id','campaign_id','date_start','date_stop','impressions','spend'])

for x in range(0,size): #o loop vai olhar dentro de cada campanha e armazenar os dados q queremos em uma tabela nova
  filtro = pd.json_normalize(trans2['insights.data'][x])
  dataset2 = pd.merge(dataset2, filtro, how='outer')


In [5]:
#Agora vamos preparar o merge para sabermos os nomes das campanhas que estamos lidando

redes = redes.rename(columns={'id':'campaign_id'})
#Agora temos um dataset oficial e pronto pra ir pra o excel!
dts = pd.merge(dataset2,redes,on='campaign_id')

Agora vamos fazer tudo novamente para as campanhas de Branding

In [6]:
#Requisitando as informações via url retirado do graph facebook api
events = requests.get("https://graph.facebook.com/v10.0/act_744828579330411/campaigns?fields=insights&time_increment=1&date_preset=this_year&access_token=EAAXwBOcBhNIBAOmxrPUCMTcDNbO8ZB8JQj8VUlCDUZBZADiQyR7wBrqizZAgkJcQqhuoZCFs7KdIhxHtJVCVBZACyAxzEzckWnwkIGZBPs27oEe6psbx3DrHxeaKNnlCZBC3aZCelDAEBzM42Ed9Jy1ptvzFhr1Hm3ZBluZCvyL1vXcVQZDZD")
#A base vai vir com campaigns_id que não possui dados internos. Tratamento dos dados abaixo
treat = json.loads(events.content)
treat2 = pd.json_normalize(treat['data']) #os dados que queremos é insights.data
treat2 = treat2.dropna(subset=['insights.data'],inplace=False).reset_index() #removendo as linhas nulas
del treat2['index'] #tratando index

#Agora vamos criar um loop para preencher uma tabela com os dados que queremos
size = treat2['id'].count()

dataset3 = pd.DataFrame(columns=['account_id','campaign_id','date_start','date_stop','impressions','spend'])

for x in range(0,size): #o loop vai olhar dentro de cada campanha e armazenar os dados q queremos em uma tabela nova
  filtro2 = pd.json_normalize(treat2['insights.data'][x])
  dataset3 = pd.merge(dataset3, filtro2, how='outer')

# **DEPLOY**

In [ ]:
#O nosso dataset é dts
!pip install --upgrade gspread
!pip install --upgrade gspread-pandas


Requirement already up-to-date: gspread in /usr/local/lib/python3.7/dist-packages (3.7.0)
Requirement already up-to-date: gspread-pandas in /usr/local/lib/python3.7/dist-packages (2.3.0)


In [ ]:
#Estabelecendo conexao com gspread

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
#sh = gc.create('FB Resume Campaign')

#Openining the new sheet
worksheet = gc.open('FB Resume Campaign')

plan1 = worksheet.worksheet('Redes Campaigns')
plan1.update([dts.columns.values.tolist()] + dts.values.tolist())

plan2 = worksheet.worksheet('Branding Campaigns')
plan2.update([dts2.columns.values.tolist()] + dts2.values.tolist())

{'spreadsheetId': '15e96GL-KH-p27H7JjW3cPSPdvMY-qa_OfM9RJZCYPXg',
 'updatedCells': 27,
 'updatedColumns': 9,
 'updatedRange': "'Branding Campaigns'!A1:I3",
 'updatedRows': 3}

# Etapa 2 - Visão Diaria da Conta

In [ ]:
#Fazendo requisicao para graphi face api para listar todas as campanhas existentes
#As campanhas mais utilizadas são de Branding e Redes, então teremos que realizar o tratamento duas vezes para cada uma
#Ao contrario das extrações anteriores, agora teremos que detalhar o quanto foi gasto diariamente por conta e por campanha
# Redes : act_625898984510034
# Branding: act_744828579330411

# "https://graph.facebook.com/v10.0/act_625898984510034/insights?date_preset=this_year&fields=spend%2Cimpressions%2Ccpm%2Ccpc&time_increment=1&limit=5000&access_token=EAAXwBOcBhNIBALI8Q3IZBZB7srwbcA112K0CMnJDN71OcVq1Ddprj4tXiilcPeyl4emZCUTNu7YipUM9nfPlCOlAgfUh3otczOh9Jtl3caZACTezmu09Dqpg0zDphekXvJRYzB24iClfem1owKrx0Fv9EYCK5qDtzxiCyyZBsk6hEmC5FMsnMHqTwS36n17BiEodZBtGLvn8bdQmGsIrMgra7k8s5Bs5qE05bvYY19j7fMg1F3CYSH"

#Custos diarios da Conta Redes
events = requests.get("https://graph.facebook.com/v10.0/act_625898984510034/insights?date_preset=this_year&time_increment=1&limit=5000&access_token=EAAXwBOcBhNIBAOmxrPUCMTcDNbO8ZB8JQj8VUlCDUZBZADiQyR7wBrqizZAgkJcQqhuoZCFs7KdIhxHtJVCVBZACyAxzEzckWnwkIGZBPs27oEe6psbx3DrHxeaKNnlCZBC3aZCelDAEBzM42Ed9Jy1ptvzFhr1Hm3ZBluZCvyL1vXcVQZDZD")
trans = json.loads(events.content)
rd_daily = pd.json_normalize(trans['data'])

#Custos diarios da Conta Branding
events = requests.get("https://graph.facebook.com/v10.0/act_744828579330411/insights?date_preset=this_year&time_increment=1&limit=5000&access_token=EAAXwBOcBhNIBAOmxrPUCMTcDNbO8ZB8JQj8VUlCDUZBZADiQyR7wBrqizZAgkJcQqhuoZCFs7KdIhxHtJVCVBZACyAxzEzckWnwkIGZBPs27oEe6psbx3DrHxeaKNnlCZBC3aZCelDAEBzM42Ed9Jy1ptvzFhr1Hm3ZBluZCvyL1vXcVQZDZD")
trans = json.loads(events.content)
bd_daily = pd.json_normalize(trans['data'])


Agora vamos jogar essas informações na nossa planilha

In [ ]:
#Estabelecendo conexao com gspread

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
#sh = gc.create('FB Resume Campaign')

#Openining the new sheet
worksheet = gc.open('FB Resume Campaign')

plan1 = worksheet.worksheet('Redes Costs')
plan1.update([rd_daily.columns.values.tolist()] + rd_daily.values.tolist())

plan2 = worksheet.worksheet('Branding Costs')
plan2.update([bd_daily.columns.values.tolist()] + bd_daily.values.tolist())

{'spreadsheetId': '15e96GL-KH-p27H7JjW3cPSPdvMY-qa_OfM9RJZCYPXg',
 'updatedCells': 170,
 'updatedColumns': 5,
 'updatedRange': "'Branding Costs'!A1:E34",
 'updatedRows': 34}

# Etapa 3 - Visao Diaria de Campanha

In [57]:
# Redes : act_625898984510034
# Branding: act_744828579330411

# "https://graph.facebook.com/v10.0/23847693270780577/insights?date_preset=this_year&time_increment=1&fields=spend%2Cimpressions%2Ccpm%2Ccpc&access_token=EAAXwBOcBhNIBALI8Q3IZBZB7srwbcA112K0CMnJDN71OcVq1Ddprj4tXiilcPeyl4emZCUTNu7YipUM9nfPlCOlAgfUh3otczOh9Jtl3caZACTezmu09Dqpg0zDphekXvJRYzB24iClfem1owKrx0Fv9EYCK5qDtzxiCyyZBsk6hEmC5FMsnMHqTwS36n17BiEodZBtGLvn8bdQmGsIrMgra7k8s5Bs5qE05bvYY19j7fMg1F3CYSH"
#Custos diarios das campanhas Rede

rd_daily = pd.DataFrame(columns=['spend','impressions','cpm','cpc','date_start','date_stop','campaign_id'])

for x in range(0,dts['campaign_id'].count()):
  campanha = dts['campaign_id'][x]
  events = requests.get("https://graph.facebook.com/v10.0/"+campanha+"/insights?time_increment=1&fields=spend%2Cimpressions%2Ccpm%2Ccpc&access_token=EAAXwBOcBhNIBAOmxrPUCMTcDNbO8ZB8JQj8VUlCDUZBZADiQyR7wBrqizZAgkJcQqhuoZCFs7KdIhxHtJVCVBZACyAxzEzckWnwkIGZBPs27oEe6psbx3DrHxeaKNnlCZBC3aZCelDAEBzM42Ed9Jy1ptvzFhr1Hm3ZBluZCvyL1vXcVQZDZD")
  trans3 = json.loads(events.content)
  filtro = pd.json_normalize(trans3['data'])
  rd_daily = pd.merge(rd_daily, filtro, how='outer')
  rd_daily['campaign_id'].fillna(dts['campaign_id'][x],inplace=True)

rd_daily = pd.merge(rd_daily,redes,on='campaign_id')

rd_daily


,campaign_id,spend,impressions,cpm,cpc,date_start,date_stop,name,daily_budget,bid_strategy
0,23847693270780577,216.61,21842,9.917132,3.800175,2021-05-05,2021-05-05,D2 - Retenção - In-App Event - Android e iOS -...,50000,LOWEST_COST_WITHOUT_CAP
1,23847693270780577,503.17,54981,9.151707,2.85892,2021-05-06,2021-05-06,D2 - Retenção - In-App Event - Android e iOS -...,50000,LOWEST_COST_WITHOUT_CAP
2,23847693270780577,536.73,64837,8.278144,2.824895,2021-05-07,2021-05-07,D2 - Retenção - In-App Event - Android e iOS -...,50000,LOWEST_COST_WITHOUT_CAP
3,23847693270780577,548.51,66966,8.190873,2.516101,2021-05-08,2021-05-08,D2 - Retenção - In-App Event - Android e iOS -...,50000,LOWEST_COST_WITHOUT_CAP
4,23847693270780577,543.67,75924,7.160713,2.255892,2021-05-09,2021-05-09,D2 - Retenção - In-App Event - Android e iOS -...,50000,LOWEST_COST_WITHOUT_CAP
...,...,...,...,...,...,...,...,...,...,...
134,23846950348250577,747.05,244117,3.060213,0.610834,2021-05-03,2021-05-03,B - Aquisição - In-App Event - Android - App C...,19700,LOWEST_COST_WITHOUT_CAP
135,23846950348250577,772.22,176082,4.38557,0.706514,2021-05-04,2021-05-04,B - Aquisição - In-App Event - Android - App C...,19700,LOWEST_COST_WITHOUT_CAP
136,23846950348250577,570.11,135725,4.200479,0.774606,2021-05-05,2021-05-05,B - Aquisição - In-App Event - Android - App C...,19700,LOWEST_COST_WITHOUT_CAP
137,23846950348250577,403.56,79463,5.07859,0.80712,2021-05-06,2021-05-06,B - Aquisição - In-App Event - Android - App C...,19700,LOWEST_COST_WITHOUT_CAP
